Let's introduce a subclass of ShippingContainer called RefrigeratedShippingContainer

In [9]:
from shipping import *

In [10]:
class RefrigeratedShippingContainer(ShippingContainer):

    @staticmethod
    def _make_bic_code(owner_code, serial):
        return iso6346.create(owner_code=owner_code, serial=str(serial).zfill(6), category='R')

In [11]:
r1 = RefrigeratedShippingContainer("MAE", 'fish')

In [12]:
r1.bic

'MAEU0013371'

'MAEU0013371' was not the desired result.  The fourth character in the BIC code is still 'U', we were expecting 'R'.  This is because in ShippingContainer.__init__() we have called _make_bic_code() through a specific class which currently takes owner_code and serial.  To get polymorpyic override behavior we need to call the static method on an instance.

First test the static method by calling _make_bic_code() directly on the base (default) class:

In [13]:
ShippingContainer._make_bic_code('MAE', 1234)

'MAEU0012349'

Then directly on the derived class:

In [14]:
RefrigeratedShippingContainer._make_bic_code('MAE', 1234)

'MAER0012347'

Now there is an 'R' for refrigeration for the fourth character.  

There are class-specific version of the static methods that have been called.  Now to create some instances.  First off, ShippingContainer or the base class:

In [15]:
c = ShippingContainer('ESC', 'textiles')

In [16]:
c ._make_bic_code('MAE', 1234)

'MAEU0012349'

The fourth character of the result is the default 'U', indicating that the base version (i.e. ShippingContainer) was called. Although an instance has been created, any instance attribue date is being ignored when the static method is invoked directly in this way.

Now instantiate the derived class (i.e. RefrigeratedShippingContainer):

In [17]:
r = RefrigeratedShippingContainer('ESC', 'peas')

In [18]:
r._make_bic_code('MAE', 1234)

'MAER0012347'

'R' is now the fourth character, indicating that hte derived class implementation is called. So we can get polymorphic dispatch of static methods only when we call the method through and instance not when we call the method through the class.  The desired result is attained by modifying __init__() in the base class to use polymorphic dispatch of the static method by calling through the instance, self:

In [19]:
#See mdofied_inheritance.ipynb & modified_shipping.py